王の作ったコード


In [5]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
# import MeCab
import random
from gensim import corpora, models
import pandas as pd
from tqdm import tqdm

# データ読み込み

2015年からのデータを辞書に使う(ちょうど5年分だし安定しているので)


In [7]:
ls_data = !ls ../data/article

In [9]:
df = pd.DataFrame()
df_len = []
n = 47
for ll in ls_data[n:n+12]:
    print(ll)
    df_buf =  pd.read_csv('../data/article/' + ll)
    df_len.append(len(df_buf))
    df = pd.concat([df, df_buf], axis = 0,  ignore_index=True)

2014-12-31 000000-content.csv
2015-01-31 000000-content.csv
2015-02-28 000000-content.csv
2015-03-31 000000-content.csv
2015-04-30 000000-content.csv
2015-05-31 000000-content.csv
2015-06-30 000000-content.csv
2015-07-31 000000-content.csv
2015-08-31 000000-content.csv
2015-09-30 000000-content.csv
2015-10-31 000000-content.csv
2015-11-30 000000-content.csv


In [10]:
# 欠損値の数と割合を求める
def missingval_table(df):
    mv_df = pd.DataFrame(df.isnull().sum())
    mv_df = mv_df.rename(columns={0:'null count'})
    mv_df['null count(%)'] = mv_df['null count']/len(df)*100
    mv_df['variation'] = df.nunique().values
    mv_df['variation(%)'] = df.nunique().values/len(df)*100
    return mv_df

missingval_table(df)

,null count,null count(%),variation,variation(%)
Unnamed: 0,0,0.000000,8492,16.198070
id,0,0.000000,52426,100.000000
article_id,0,0.000000,52426,100.000000
title,0,0.000000,52376,99.904627
created_at,0,0.000000,52334,99.824515
likes_count,0,0.000000,758,1.445847
tags_str,0,0.000000,26066,49.719605
url,0,0.000000,52426,100.000000
tokens,3,0.005722,52038,99.259909


In [70]:
df_len

[3694, 3792, 3846, 3638, 3989, 3946, 4283, 4056, 3953, 4236, 4501, 8492]

In [11]:
df['tokens'] = df['tokens'].replace(np.nan, "", regex=True)
missingval_table(df)

,null count,null count(%),variation,variation(%)
Unnamed: 0,0,0.0,8492,16.198070
id,0,0.0,52426,100.000000
article_id,0,0.0,52426,100.000000
title,0,0.0,52376,99.904627
created_at,0,0.0,52334,99.824515
likes_count,0,0.0,758,1.445847
tags_str,0,0.0,26066,49.719605
url,0,0.0,52426,100.000000
tokens,0,0.0,52039,99.261817


## 辞書を作成する

In [1]:
import unicodedata

def normalize(text):
    normalized_text = normalize_unicode(text)
    normalized_text = lower_text(normalized_text)
    return normalized_text


def lower_text(text):
    return text.lower()


def normalize_unicode(text, form='NFKC'):
    normalized_text = unicodedata.normalize(form, text)
    return normalized_text

In [7]:
ls_data = !ls ../data/article

def make_dict(year, dpath):
    df = pd.DataFrame()
    df_len = []
    for ll in ls_data:
        if ll.find(str(year)):
            continue
        print(ll)
        df_buf =  pd.read_csv("../data/article/" + ll)
        df_len.append(len(df_buf))
        df = pd.concat([df, df_buf], axis = 0,  ignore_index=True)

    df['tokens'] = df['tokens'].replace(np.nan, "", regex=True)
    print(df_len)
    share_list = []
    for sh in df["tokens"]:
        sh_no = normalize(sh)
    #     print(share)
        if type(sh_no) is str:        
            share_list.append([s for s in sh_no.split(" ") if len(s) > 1])
        else:
            share_list.append([])

    d = corpora.Dictionary(share_list)
    print("num pos : " + str(d.num_pos))
    d.save(dpath + "dict{}.dict".format(year))
    return d

In [135]:
dpath = "buf_dic/"
for i in range(2014, 2020):
    make_dict(i, dpath)

2014-01-31 000000-content.csv
2014-02-28 000000-content.csv
2014-03-31 000000-content.csv
2014-04-30 000000-content.csv
2014-05-31 000000-content.csv
2014-06-30 000000-content.csv
2014-07-31 000000-content.csv
2014-08-31 000000-content.csv
2014-09-30 000000-content.csv
2014-10-31 000000-content.csv
2014-11-30 000000-content.csv
2014-12-31 000000-content.csv
[2253, 2516, 2792, 2948, 3155, 3321, 3070, 3164, 3161, 3239, 5411, 3694]


adding document #0 to Dictionary(0 unique tokens: [])
adding document #10000 to Dictionary(64967 unique tokens: ['...', '02', '14', '2014', '_・']...)
adding document #20000 to Dictionary(100690 unique tokens: ['...', '02', '14', '2014', '_・']...)
adding document #30000 to Dictionary(132522 unique tokens: ['...', '02', '14', '2014', '_・']...)
built Dictionary(165319 unique tokens: ['...', '02', '14', '2014', '_・']...) from 38724 documents (total 7588124 corpus positions)
saving Dictionary object under buf_dic/dict2014.dict, separately None
saved buf_dic/dict2014.dict


num pos : 7588124
2015-01-31 000000-content.csv
2015-02-28 000000-content.csv
2015-03-31 000000-content.csv
2015-04-30 000000-content.csv
2015-05-31 000000-content.csv
2015-06-30 000000-content.csv
2015-07-31 000000-content.csv
2015-08-31 000000-content.csv
2015-09-30 000000-content.csv
2015-10-31 000000-content.csv
2015-11-30 000000-content.csv
2015-12-31 000000-content.csv
[3792, 3846, 3638, 3989, 3946, 4283, 4056, 3953, 4236, 4501, 8492, 4868]


adding document #0 to Dictionary(0 unique tokens: [])
adding document #10000 to Dictionary(77917 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...)
adding document #20000 to Dictionary(117490 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...)
adding document #30000 to Dictionary(150296 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...)
adding document #40000 to Dictionary(182052 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...)
adding document #50000 to Dictionary(224228 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...)
built Dictionary(235725 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...) from 53600 documents (total 12332760 corpus positions)
saving Dictionary object under buf_dic/dict2015.dict, separately None
saved buf_dic/dict2015.dict


num pos : 12332760
2016-01-31 000000-content.csv
2016-02-29 000000-content.csv
2016-03-31 000000-content.csv
2016-04-30 000000-content.csv
2016-05-31 000000-content.csv
2016-06-30 000000-content.csv
2016-07-31 000000-content.csv
2016-08-31 000000-content.csv
2016-09-30 000000-content.csv
2016-10-31 000000-content.csv
2016-11-30 000000-content.csv
2016-12-31 000000-content.csv
[5117, 5316, 4807, 4819, 4812, 4863, 4604, 4897, 4992, 5056, 10584, 5112]


...

num pos : 19840244
2018-01-31 000000-content.csv
2018-02-28 000000-content.csv
2018-03-31 000000-content.csv
2018-04-30 000000-content.csv
2018-05-31 000000-content.csv
2018-06-30 000000-content.csv
2018-07-31 000000-content.csv
2018-08-31 000000-content.csv
2018-09-30 000000-content.csv
2018-10-31 000000-content.csv
2018-11-30 000000-content.csv
2018-12-31 000000-content.csv
[5958, 7180, 7179, 7296, 7364, 7099, 7231, 7296, 8439, 8246, 14399, 8287]


adding document #0 to Dictionary(0 unique tokens: [])
adding document #10000 to Dictionary(96000 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #20000 to Dictionary(142057 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #30000 to Dictionary(180254 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #40000 to Dictionary(216782 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #50000 to Dictionary(252695 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #60000 to Dictionary(284669 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #70000 to Dictionary(321619 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #80000 to Dictionary(359732 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
adding document #90000 to Dictionary(396108 unique tokens: ['("-"', '10', '13.', '16', '2.0']...)
built Dictionary(412553 unique tokens: ['("-"', '10', '13.', '16'

num pos : 29353182
2019-01-31 000000-content.csv
2019-02-28 000000-content.csv
2019-03-31 000000-content.csv
2019-04-30 000000-content.csv
2019-05-31 000000-content.csv
2019-06-30 000000-content.csv
2019-07-31 000000-content.csv
2019-08-31 000000-content.csv
2019-09-30 000000-content.csv
2019-10-31 000000-content.csv
2019-11-30 000000-content.csv
[8225, 9322, 9481, 10056, 9899, 9762, 9508, 9839, 10453, 10656, 63]


adding document #0 to Dictionary(0 unique tokens: [])
adding document #10000 to Dictionary(95216 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
adding document #20000 to Dictionary(153287 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
adding document #30000 to Dictionary(194769 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
adding document #40000 to Dictionary(231346 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
adding document #50000 to Dictionary(271118 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
adding document #60000 to Dictionary(304167 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
adding document #70000 to Dictionary(334547 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
adding document #80000 to Dictionary(367385 unique tokens: ['base

num pos : 28105345


# 辞書の結合
2014-2018を結合する

In [136]:
ls_dict = !ls buf_dic

def concat_dict(name, no_below=5, no_above=0.2, save=False):
    d_all = corpora.Dictionary()
    for di in ls_dict:
        d = corpora.Dictionary.load("buf_dic/" + di)
        d.filter_extremes(no_below=no_below, no_above=no_above)
        d.compactify()
        print(len(d))
        d_all.merge_with(d)
        print("d_all: " + str(len(d_all)))
    d_all.compactify()
    print(len(d_all))
    if save:
        d_all.save("../data/dictionary/dict{}.dict".format(name))
    return d_all
    
d = concat_dict("final_6years_14-19", save=True)

loading Dictionary object from buf_dic/dict2014.dict
loaded buf_dic/dict2014.dict
discarding 134014 tokens: [('_・', 1), ('stringjoiner', 1), ('これ', 13504), ('たら', 10308), ('できる', 12825), ('ない', 22537), ('なる', 11490), ('まし', 14577), ('もの', 9893), ('cvfold', 1)]...
keeping 31305 tokens which were in no less than 5 and no more than 7744 (=20.0%) documents
resulting dictionary: Dictionary(31305 unique tokens: ['...', '02', '14', '2014', 'api']...)
loading Dictionary object from buf_dic/dict2015.dict
loaded buf_dic/dict2015.dict


31305
d_all: 31305


discarding 193515 tokens: [('ある', 25423), ('いる', 29348), ('から', 28996), ('こと', 30092), ('この', 20295), ('これ', 19824), ('する', 41622), ('そう', 10936), ('たい', 18400), ('たら', 15182)]...
keeping 42210 tokens which were in no less than 5 and no more than 10720 (=20.0%) documents
resulting dictionary: Dictionary(42210 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...)
loading Dictionary object from buf_dic/dict2016.dict
loaded buf_dic/dict2016.dict


42210
d_all: 44881


discarding 249290 tokens: [('あり', 24059), ('ある', 32653), ('いる', 37833), ('から', 37312), ('これ', 24836), ('する', 51495), ('その', 19687), ('ため', 23866), ('だけ', 20951), ('でき', 29232)]...
keeping 51109 tokens which were in no less than 5 and no more than 12995 (=20.0%) documents
resulting dictionary: Dictionary(51109 unique tokens: ['1号', '2号', 'a.', 'basebackup', 'con']...)
loading Dictionary object from buf_dic/dict2017.dict
loaded buf_dic/dict2017.dict


...

55064
d_all: 66394


discarding 344145 tokens: [('("-"', 3), ('あり', 39903), ('ある', 52260), ('いる', 59391), ('から', 59130), ('ここ', 25066), ('こと', 62454), ('これ', 39463), ('する', 79309), ('そう', 23192)]...
keeping 68408 tokens which were in no less than 5 and no more than 19194 (=20.0%) documents
resulting dictionary: Dictionary(68408 unique tokens: ['10', '13.', '16', '2.0', '4r']...)
loading Dictionary object from buf_dic/dict2019.dict


68408
d_all: 80216


loaded buf_dic/dict2019.dict
discarding 347003 tokens: [('ある', 51790), ('から', 58097), ('ここ', 23671), ('こと', 62500), ('この', 42813), ('これ', 38508), ('する', 79379), ('それ', 20566), ('できる', 41546), ('ない', 63183)]...
keeping 67535 tokens which were in no less than 5 and no more than 19452 (=20.0%) documents
resulting dictionary: Dictionary(67535 unique tokens: ['base64', 'buffer', 'cloudfunctions', 'cloudstorage', 'post']...)
saving Dictionary object under ../data/dictionary/dictfinal_6years_14-19.dict, separately None
saved ../data/dictionary/dictfinal_6years_14-19.dict


67535
d_all: 88843
88843


In [63]:
d_one = corpora.Dictionary.load("buf_dic/dict2018.dict")
d_one.filter_extremes(no_below=5, no_above=0.2)
d_one.save("../data/dictionary/dict2018.dict")

loading Dictionary object from buf_dic/dict2018.dict
loaded buf_dic/dict2018.dict
discarding 344145 tokens: [('("-"', 3), ('あり', 39903), ('ある', 52260), ('いる', 59391), ('から', 59130), ('ここ', 25066), ('こと', 62454), ('これ', 39463), ('する', 79309), ('そう', 23192)]...
keeping 68408 tokens which were in no less than 5 and no more than 19194 (=20.0%) documents
resulting dictionary: Dictionary(68408 unique tokens: ['10', '13.', '16', '2.0', '4r']...)
saving Dictionary object under ../data/dictionary/dict2018.dict, separately None
saved ../data/dictionary/dict2018.dict


分かち書き後の処理

・1文字なものを削除

・すべて小文字に変更



データベースから一年分のデータを読み込む

In [26]:
# d.filter_extremes(no_below=5, no_above=0.2)
# # nob_below以下の個数の単語を無視
# # no_aboveの割合以上に出てくる単語を無視
# d.compactify()
# # idを振り直してコンパクトにする。

# 確認

In [143]:
d = d_all

In [15]:
def check(dic): 
    "消えてほしくない単語が消えないようにする"
    print("len: " + str(len(d)))
    print(d[1289])
    print("python id:",d.token2id["python"])
    print("C++ id:",d.token2id["c++"])
    print(d.doc2bow(["探索","ノード"]))

In [20]:
check(d)

len: 80216
1gb
python id: 89
C++ id: 227
[(996, 1), (6800, 1)]


In [23]:
d = concat_dict("2014-2018", no_below=100, no_above=0.1)
check(d)

4578
d_all: 4578
6256
d_all: 6293
7917
d_all: 8013
8613
d_all: 8902
11079
d_all: 11266
11266
len: 11266
document
python id: 57
C++ id: 157
[(735, 1), (3558, 1)]


In [134]:
print(d.doc2bow(["ビルド"]))
print(d.token2id["ビルド"])
d.dfs[d.token2id["ブチ"]]
lda5.show_topics(num_topics=5, num_words=1000, log=False, formatted=True)

[(735, 1)]
735


[(13,
  '0.093*"取得" + 0.059*"api" + 0.043*"情報" + 0.040*"認証" + 0.034*"アクセス" + 0.033*"id" + 0.025*"トークン" + 0.020*"ログイン" + 0.016*"ユーザ" + 0.015*"url" + 0.014*"ユーザー" + 0.013*"登録" + 0.012*"token" + 0.010*"key" + 0.010*"json" + 0.010*"アカウント" + 0.009*"ポイント" + 0.009*"発行" + 0.008*"エンド" + 0.008*"アプリ" + 0.007*"下記" + 0.007*"保存" + 0.007*"アプリケーション" + 0.006*"app" + 0.006*"権限" + 0.006*"アクセストークン" + 0.006*"oauth" + 0.006*"キー" + 0.006*"認可" + 0.006*"生成" + 0.005*"access" + 0.005*"http" + 0.005*"":" + 0.005*"admin" + 0.005*"ましょ" + 0.005*"user" + 0.005*"一覧" + 0.005*"有効" + 0.005*"パラメータ" + 0.004*"リダイレクト" + 0.004*"ドキュメント" + 0.004*"パスワード" + 0.004*"すれ" + 0.004*"secret" + 0.004*"サービス" + 0.004*"type" + 0.004*"client" + 0.003*"ad" + 0.003*"例えば" + 0.003*"検証" + 0.003*"返っ" + 0.003*"注意" + 0.003*"jwt" + 0.003*"request" + 0.003*"セッション" + 0.003*"によって" + 0.003*"許可" + 0.003*"github" + 0.003*"ログアウト" + 0.003*"エラー" + 0.003*"cli" + 0.003*"おき" + 0.003*"でしょ" + 0.003*"内容" + 0.003*"クッキー" + 0.003*"フロー" + 0.003*"cookie" + 0.003*"リクエスト"

In [147]:
dic_num = len(d)
dic_num

88229

In [148]:
d[0]

'issue'

In [127]:
aa = [(d[t[0]],t[1])for t in sorted(d.dfs.items(),key=lambda x:x[1],reverse=True)]

In [128]:
print(len(aa))
aa[len(aa)-60000:]

80216


[('sit', 108),
 ('synchronized', 108),
 ('辻褄', 108),
 ('middleman', 108),
 ('擬似乱数', 108),
 ('明け', 108),
 ('あれから', 108),
 ('目前', 108),
 ('共役', 108),
 ('stay', 108),
 ('ioexception', 108),
 ('見つけよ', 108),
 ('amqp', 108),
 ('けれ', 108),
 ('1023', 108),
 ('こねくり', 108),
 ('instanceof', 108),
 ('ときの', 108),
 ('familiar', 108),
 ('ややこしかっ', 108),
 ('20k', 108),
 ('叶え', 108),
 ('3色', 108),
 ('ブルー', 108),
 ('リラックス', 108),
 ('穏やか', 108),
 ('覚え方', 108),
 ('とっとと', 108),
 ('係る', 108),
 ('かれ', 108),
 ('短縮url', 108),
 ('カレー', 108),
 ('最早', 108),
 ('あいうえお', 108),
 ('baz', 108),
 ('rain', 108),
 ('なぜだか', 108),
 ('editorconfig', 108),
 ('x21', 108),
 ('合図', 108),
 ('charles', 108),
 ('anyone', 108),
 ('ネットワークアダプター', 108),
 ('真逆', 108),
 ('辞める', 108),
 ('minus', 108),
 ('spider', 108),
 ('formatted', 108),
 ('インチ', 108),
 ('よび', 108),
 ('一種類', 108),
 ('vccw', 108),
 ('kv', 108),
 ('持ち上げ', 108),
 ('仮装', 108),
 ('construction', 108),
 ('performing', 108),
 ('馴染み深い', 108),
 ('organizations', 108),
 ('付け替える', 

# コーパスを作成する

# bow


In [30]:
# corpus = [d.doc2bow(w) for w in docs]
# test_size = int(len(corpus) * 0.1)
# test_corpus = corpus[:test_size]
# train_corpus= corpus[test_size:]
# print("test size: ", len(test_corpus))
# print("train size: ", len(train_corpus))

In [50]:
from tqdm._tqdm_notebook import tqdm_notebook as tdqm

def make_share_list(df):
    "分かち書きをリストにする　いち文字の単語は除く"
    share_list = []
    for sh in tdqm(df["tokens"]):
        sh_no = normalize(sh)
    #     print(share)
        if type(sh_no) is str:        
            share_list.append([s for s in sh_no.split(" ") if len(s) > 1])
        else:
            share_list.append([])
    return share_list
    
def make_bow_corpus(d, df):
    "bowのコーパスを作成する"
    df['tokens'] = df['tokens'].replace(np.nan, "", regex=True)
    print(df_len)
    share_list = make_share_list(df)
    corpus = [d.doc2bow(w) for w in share_list]
    return corpus

In [138]:
!ls ../data/article | egrep "2019|2018|2017|2016|2015"

2015-01-31 000000-content.csv
2015-02-28 000000-content.csv
2015-03-31 000000-content.csv
2015-04-30 000000-content.csv
2015-05-31 000000-content.csv
2015-06-30 000000-content.csv
2015-07-31 000000-content.csv
2015-08-31 000000-content.csv
2015-09-30 000000-content.csv
2015-10-31 000000-content.csv
2015-11-30 000000-content.csv
2015-12-31 000000-content.csv
2016-01-31 000000-content.csv
2016-02-29 000000-content.csv
2016-03-31 000000-content.csv
2016-04-30 000000-content.csv
2016-05-31 000000-content.csv
2016-06-30 000000-content.csv
2016-07-31 000000-content.csv
2016-08-31 000000-content.csv
2016-09-30 000000-content.csv
2016-10-31 000000-content.csv
2016-11-30 000000-content.csv
2016-12-31 000000-content.csv
2017-01-31 000000-content.csv
2017-02-28 000000-content.csv
2017-03-31 000000-content.csv
2017-04-30 000000-content.csv
2017-05-31 000000-content.csv
2017-06-30 000000-content.csv
2017-07-31 000000-content.csv
2017-08-

In [137]:
ls2019 = !ls ../data/article | egrep "2019|2018|2017|2016|2015"

df_2019 = pd.DataFrame()
df_len = []
for ll in ls2019:
    df_buf =  pd.read_csv('../data/article/' + ll)
    df_len.append(len(df_buf))
    df_2019 = pd.concat([df_2019, df_buf], axis = 0,  ignore_index=True)

In [139]:
corpus_5years = make_bow_corpus(d, df_2019)
corpora.MmCorpus.serialize("../data/corpus/bow_copus_from_dict2015-2019.mm", corpus_5years)

[3792, 3846, 3638, 3989, 3946, 4283, 4056, 3953, 4236, 4501, 8492, 4868, 5117, 5316, 4807, 4819, 4812, 4863, 4604, 4897, 4992, 5056, 10584, 5112, 4901, 5274, 5222, 4979, 4871, 4707, 4713, 5030, 5447, 5948, 11777, 6263, 5958, 7180, 7179, 7296, 7364, 7099, 7231, 7296, 8439, 8246, 14399, 8287, 8225, 9322, 9481, 10056, 9899, 9762, 9508, 9839, 10453, 10656, 63]


storing corpus in Matrix Market format to ../data/corpus/bow_copus_from_dict2015-2019.mm
saving sparse matrix to ../data/corpus/bow_copus_from_dict2015-2019.mm
PROGRESS: saving document #0
PROGRESS: saving document #1000
PROGRESS: saving document #2000
PROGRESS: saving document #3000
PROGRESS: saving document #4000
PROGRESS: saving document #5000
PROGRESS: saving document #6000
PROGRESS: saving document #7000
PROGRESS: saving document #8000
PROGRESS: saving document #9000
PROGRESS: saving document #10000
PROGRESS: saving document #11000
PROGRESS: saving document #12000
PROGRESS: saving document #13000
PROGRESS: saving document #14000
PROGRESS: saving document #15000
PROGRESS: saving document #16000
PROGRESS: saving document #17000
PROGRESS: saving document #18000
PROGRESS: saving document #19000
PROGRESS: saving document #20000
PROGRESS: saving document #21000
PROGRESS: saving document #22000
PROGRESS: saving document #23000
PROGRESS: saving document #24000
PROGRESS: saving document #2

KeyboardInterrupt: 

In [163]:
corpus_oneyear = make_bow_corpus(2019, d_one)
corpora.MmCorpus.serialize("data/corpus/bow_copus_from_dict2018.mm", corpus_oneyear)

2019-01-31 000000-content.csv
2019-02-28 000000-content.csv
2019-03-31 000000-content.csv
2019-04-30 000000-content.csv
2019-05-31 000000-content.csv
2019-06-30 000000-content.csv
2019-07-31 000000-content.csv
2019-08-31 000000-content.csv
2019-09-30 000000-content.csv
2019-10-31 000000-content.csv
2019-11-30 000000-content.csv
[8225, 9322, 9481, 10056, 9899, 9762, 9508, 9839, 10453, 10656, 63]


# tfidf

In [103]:
# tfidf
import gensim
import pickle
# # tfidf modelの生成
tfidf = gensim.models.TfidfModel(corpus_5years)
tfidf.save('corpus/model_5years.tfidf')

# make corpus_tfidf
corpus_tfidf = tfidf[corpus_5years]

# save corpus_tfidf
with open("corpus/corpus_tfidf.pkl",'wb') as f:
    pickle.dump(corpus_tfidf,f)

collecting document frequencies
PROGRESS: processing document #0
PROGRESS: processing document #10000
PROGRESS: processing document #20000
PROGRESS: processing document #30000
PROGRESS: processing document #40000
PROGRESS: processing document #50000
PROGRESS: processing document #60000
PROGRESS: processing document #70000
PROGRESS: processing document #80000
PROGRESS: processing document #90000
calculating IDF weights for 97264 documents and 11266 features (8618532 matrix non-zeros)
saving TfidfModel object under corpus/model_5years.tfidf, separately None
saved corpus/model_5years.tfidf


# やる手順
一年分で辞書を作成する

一月分でコーパスを作成する

1. tfidfとbowで比べる

2.辞書についてno_belowなどの変数を変更して比較する

In [141]:
ls2019 = !ls ../data/article | grep 2019

df_2019 = pd.DataFrame()
df_len = []
for ll in ls2019:
    df_buf =  pd.read_csv('../data/article/' + ll)
    df_len.append(len(df_buf))
    df_2019 = pd.concat([df_2019, df_buf], axis = 0,  ignore_index=True)

In [142]:
corpus_5years = make_bow_corpus(d, df_2019)
corpora.MmCorpus.serialize("../data/corpus/bow_copus_from_dict2015-2019.mm", corpus_5years)

[8225, 9322, 9481, 10056, 9899, 9762, 9508, 9839, 10453, 10656, 63]


storing corpus in Matrix Market format to ../data/corpus/bow_copus_from_dict2015-2019.mm
saving sparse matrix to ../data/corpus/bow_copus_from_dict2015-2019.mm
PROGRESS: saving document #0
PROGRESS: saving document #1000
PROGRESS: saving document #2000
PROGRESS: saving document #3000
PROGRESS: saving document #4000
PROGRESS: saving document #5000
PROGRESS: saving document #6000
PROGRESS: saving document #7000
PROGRESS: saving document #8000
PROGRESS: saving document #9000
PROGRESS: saving document #10000
PROGRESS: saving document #11000
PROGRESS: saving document #12000
PROGRESS: saving document #13000
PROGRESS: saving document #14000
PROGRESS: saving document #15000
PROGRESS: saving document #16000
PROGRESS: saving document #17000
PROGRESS: saving document #18000
PROGRESS: saving document #19000
PROGRESS: saving document #20000
PROGRESS: saving document #21000
PROGRESS: saving document #22000
PROGRESS: saving document #23000
PROGRESS: saving document #24000
PROGRESS: saving document #2

In [143]:
lda_final = models.ldamodel.LdaModel(corpus=corpus_5years, id2word=d, num_topics=60, passes=20)

using symmetric alpha at 0.016666666666666666
using symmetric eta at 0.016666666666666666
using serial LDA version on this node
running online (multi-pass) LDA training, 60 topics, 20 passes over the supplied corpus of 97264 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #37 (0.017): 0.005*"64" + 0.004*"使用" + 0.004*"利用" + 0.004*"要素" + 0.004*"bit" + 0.004*"コマンド" + 0.004*"入力" + 0.004*"環境" + 0.004*"処理" + 0.003*"32"
topic #4 (0.017): 0.006*"コマンド" + 0.005*"環境" + 0.004*"データ" + 0.004*"コード" + 0.004*"使用" + 0.004*"削除" + 0.004*"可能" + 0.003*"実装" + 0.003*"to" + 0.003*"動作"
topic #29 (0.017): 0.005*"データ" + 0.004*"使用" + 0.004*"コマンド" + 0.004*"変更" + 0.004*"入力" + 0.004*"情報" + 0.004*"それ" + 0.003*"選択" + 0.003*"環境" + 0.003*"状態"
topic #46 (0.017): 0.004*"プロジェクト" + 0.004*"データ" + 0.004*"環境" +

topic #8 (0.017): 0.020*"to" + 0.018*"with" + 0.017*"and" + 0.015*")," + 0.015*"by" + 0.014*")." + 0.014*"on" + 0.013*"the" + 0.011*"data" + 0.011*"of"
topic #41 (0.017): 0.008*"てる" + 0.007*"検索" + 0.007*"記事" + 0.006*"とか" + 0.005*"command" + 0.005*"感じ" + 0.005*"そう" + 0.004*"って" + 0.004*"でも" + 0.004*"go"
topic #11 (0.017): 0.056*"処理" + 0.014*"例外" + 0.011*"発生" + 0.011*"プロセス" + 0.009*"条件" + 0.009*"スレッド" + 0.008*"ハッシュ" + 0.008*"非同期" + 0.008*"()" + 0.006*"終了"
topic diff=0.520816, rho=0.333333
-11.956 per-word bound, 3972.8 perplexity estimate based on a held-out corpus of 2000 documents with 403508 words
PROGRESS: pass 0, at document #20000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #12 (0.017): 0.033*"flutter" + 0.020*"":" + 0.020*".," + 0.016*"null" + 0.015*""," + 0.014*"gae" + 0.008*"ウィジェット" + 0.008*"flow" + 0.007*"point" + 0.007*"builder"
topic #34 (0.017): 0.049*"インスタンス" + 0.028*"ec2" + 0.015*"aws" + 0.015*"vpc" + 0.010*"amazon" + 0.009*"接続" + 0.008*

PROGRESS: pass 0, at document #36000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #8 (0.017): 0.021*"by" + 0.019*"with" + 0.017*"to" + 0.013*"data" + 0.012*"for" + 0.012*"on" + 0.012*"and" + 0.011*")," + 0.010*"in" + 0.010*"of"
topic #44 (0.017): 0.071*"画像" + 0.022*"学習" + 0.020*"gpu" + 0.019*"モデル" + 0.015*"生成" + 0.014*"python" + 0.013*"tensorflow" + 0.011*"分類" + 0.011*"ラベル" + 0.010*"opencv"
topic #14 (0.017): 0.029*"描画" + 0.025*"アニメーション" + 0.012*"頂点" + 0.011*"svg" + 0.011*"画像" + 0.011*"canvas" + 0.010*"gulp" + 0.009*"カメラ" + 0.008*"()" + 0.008*"on"
topic #16 (0.017): 0.034*"データ" + 0.023*"計算" + 0.016*"結果" + 0.009*"時間" + 0.007*"今回" + 0.007*"問題" + 0.007*"グラフ" + 0.006*"より" + 0.005*"られ" + 0.005*"用い"
topic #54 (0.017): 0.051*"テスト" + 0.029*"rails" + 0.022*"ruby" + 0.012*"id" + 0.010*"user" + 0.010*"モデル" + 0.009*"テーブル" + 0.009*"gem" + 0.008*"エラー" + 0.008*"記述"
topic diff=0.202497, rho=0.235702
PROGRESS: pass 0, at document #38000/97264
merging changes from 2000

topic #18 (0.017): 0.141*"ログ" + 0.122*"-1" + 0.098*"-2" + 0.052*"-3" + 0.028*"-4" + 0.028*"出力" + 0.028*"log" + 0.023*"elasticsearch" + 0.019*"graphql" + 0.015*"filter"
topic #38 (0.017): 0.065*"アプリ" + 0.028*"app" + 0.026*"アカウント" + 0.024*"slack" + 0.022*"cloud" + 0.020*"google" + 0.020*"heroku" + 0.018*"通知" + 0.018*"->" + 0.014*"デプロイ"
topic diff=0.115675, rho=0.196116
PROGRESS: pass 0, at document #54000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #34 (0.017): 0.068*"インスタンス" + 0.044*"aws" + 0.041*"ec2" + 0.021*"amazon" + 0.018*"vpc" + 0.017*"ip" + 0.015*"グループ" + 0.011*"リージョン" + 0.010*"rds" + 0.010*"タイプ"
topic #48 (0.017): 0.023*"公開鍵" + 0.022*"暗号" + 0.018*"秘密鍵" + 0.017*"key" + 0.017*"署名" + 0.015*"ie" + 0.014*"暗号化" + 0.013*"方式" + 0.013*"印刷" + 0.010*"又は"
topic #29 (0.017): 0.030*"起動" + 0.028*"windows" + 0.020*"ubuntu" + 0.018*"接続" + 0.018*"os" + 0.017*"linux" + 0.014*"pi" + 0.014*"pc" + 0.012*"ダウンロード" + 0.012*"raspberry"
topic #23 (0.017): 0.079*"配列" + 0

topic #38 (0.017): 0.064*"アプリ" + 0.045*"cloud" + 0.029*"google" + 0.026*"アカウント" + 0.026*"app" + 0.024*"通知" + 0.024*"デプロイ" + 0.019*"slack" + 0.017*"heroku" + 0.015*"gcp"
topic #15 (0.017): 0.012*"使用" + 0.010*"システム" + 0.009*"によって" + 0.008*"より" + 0.008*"または" + 0.008*"すべて" + 0.007*"べき" + 0.007*"可能" + 0.007*"単語" + 0.007*"提供"
topic #47 (0.017): 0.017*"開発" + 0.007*"システム" + 0.007*"チーム" + 0.007*"設計" + 0.006*"サービス" + 0.006*"機能" + 0.006*"技術" + 0.005*"時間" + 0.005*"企業" + 0.005*"運用"
topic #44 (0.017): 0.094*"画像" + 0.021*"学習" + 0.020*"モデル" + 0.017*"gpu" + 0.015*"tensorflow" + 0.012*"検出" + 0.012*"生成" + 0.012*"ラベル" + 0.010*"データセット" + 0.010*"出力"
topic #32 (0.017): 0.104*"メール" + 0.062*"->" + 0.060*");" + 0.045*"セル" + 0.045*"送信" + 0.025*"メールアドレス" + 0.025*"圧縮" + 0.020*"==" + 0.014*"");" + 0.013*"viewcontroller"
topic diff=0.075021, rho=0.169031
PROGRESS: pass 0, at document #72000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #16 (0.017): 0.029*"データ" + 0.020*"計算" + 0.015*"

...

topic #47 (0.017): 0.019*"開発" + 0.009*"サービス" + 0.009*"システム" + 0.008*"チーム" + 0.007*"設計" + 0.007*"技術" + 0.007*"運用" + 0.007*"参加" + 0.007*"管理" + 0.006*"ツール"
topic diff=0.020580, rho=0.120708
PROGRESS: pass 19, at document #28000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #18 (0.017): 0.133*"-1" + 0.133*"-2" + 0.131*"ログ" + 0.104*"出力" + 0.049*"-3" + 0.049*"2007" + 0.037*"log" + 0.031*"-4" + 0.025*"フィルタ" + 0.024*"elasticsearch"
topic #10 (0.017): 0.015*"とか" + 0.013*"いい" + 0.013*"感じ" + 0.012*"そう" + 0.010*"って" + 0.010*"けど" + 0.010*"てる" + 0.009*"なら" + 0.009*"やっ" + 0.009*"たり"
topic #59 (0.017): 0.131*"アプリ" + 0.053*"android" + 0.030*"ios" + 0.018*"xcode" + 0.016*"対応" + 0.015*"kotlin" + 0.014*"端末" + 0.014*"swift" + 0.014*"app" + 0.014*"開発"
topic #38 (0.017): 0.051*"google" + 0.042*"cloud" + 0.028*"デプロイ" + 0.027*"slack" + 0.026*"サービス" + 0.024*"firebase" + 0.022*"通知" + 0.022*"アカウント" + 0.022*"heroku" + 0.020*"bot"
topic #52 (0.017): 0.052*"タグ" + 0.049*"要素" + 0.038*

topic #27 (0.017): 0.057*"_{" + 0.047*"$\" + 0.029*"}$" + 0.026*")$" + 0.018*"frac" + 0.015*"{\" + 0.015*"$$" + 0.013*"}\" + 0.012*"theta" + 0.012*"}{"
topic #1 (0.017): 0.048*"関数" + 0.041*"コード" + 0.027*"実装" + 0.018*"処理" + 0.013*"定義" + 0.012*"引数" + 0.011*"今回" + 0.009*"ここ" + 0.008*"について" + 0.008*"部分"
topic #4 (0.017): 0.071*"string" + 0.071*"()" + 0.024*".s" + 0.023*"修正" + 0.020*"true" + 0.020*"if" + 0.017*"::" + 0.016*"false" + 0.014*"return" + 0.013*"int"
topic #51 (0.017): 0.060*"unity" + 0.042*"ゲーム" + 0.039*"動画" + 0.030*"オブジェクト" + 0.027*"再生" + 0.022*"3d" + 0.020*"フレーム" + 0.019*"音声" + 0.018*"シーン" + 0.017*"スクリプト"
topic diff=0.014322, rho=0.120708
PROGRESS: pass 19, at document #46000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #29 (0.017): 0.056*"windows" + 0.036*"起動" + 0.033*"os" + 0.029*"linux" + 0.029*"ubuntu" + 0.029*"環境" + 0.017*"pc" + 0.017*"ダウンロード" + 0.015*"mac" + 0.014*"10"
topic #4 (0.017): 0.071*"()" + 0.066*"string" + 0.024*".s" + 0.022*"

topic diff=0.017118, rho=0.120708
PROGRESS: pass 19, at document #62000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #47 (0.017): 0.020*"開発" + 0.011*"システム" + 0.009*"サービス" + 0.008*"チーム" + 0.008*"設計" + 0.007*"ツール" + 0.007*"技術" + 0.006*"運用" + 0.006*"管理" + 0.006*"参加"
topic #1 (0.017): 0.048*"関数" + 0.040*"コード" + 0.027*"実装" + 0.018*"処理" + 0.013*"定義" + 0.012*"引数" + 0.011*"今回" + 0.009*"ここ" + 0.008*"について" + 0.008*"プログラム"
topic #30 (0.017): 0.263*"python" + 0.025*"ver" + 0.024*"pip" + 0.022*"jupyter" + 0.020*"anaconda" + 0.020*"仮想" + 0.018*"py" + 0.017*"notebook" + 0.016*"環境" + 0.014*"モジュール"
topic #21 (0.017): 0.062*"接続" + 0.034*"ssh" + 0.023*"ポート" + 0.022*"サーバ" + 0.020*"起動" + 0.017*"ip" + 0.015*"パスワード" + 0.015*"アクセス" + 0.013*"ログイン" + 0.013*"server"
topic #15 (0.017): 0.012*"可能" + 0.012*"使用" + 0.012*"について" + 0.011*"機能" + 0.010*"によって" + 0.008*"それ" + 0.008*"利用" + 0.008*"より" + 0.007*"情報" + 0.006*"により"
topic diff=0.016388, rho=0.120708
PROGRESS: pass 19, at documen

topic #28 (0.017): 0.074*"テーブル" + 0.052*"データベース" + 0.050*"db" + 0.038*"sql" + 0.035*"データ" + 0.024*"レコード" + 0.023*"mysql" + 0.016*"クエリ" + 0.016*"カラム" + 0.013*"database"
topic #14 (0.017): 0.038*"位置" + 0.029*"座標" + 0.027*"描画" + 0.024*"アニメーション" + 0.019*"移動" + 0.019*"方向" + 0.017*"回転" + 0.012*"配置" + 0.011*"頂点" + 0.009*"box"
topic diff=0.014266, rho=0.120708
-11.835 per-word bound, 3653.6 perplexity estimate based on a held-out corpus of 2000 documents with 391963 words
PROGRESS: pass 19, at document #80000/97264
merging changes from 2000 documents into a model of 97264 documents
topic #7 (0.017): 0.016*"自分" + 0.013*"理解" + 0.009*"プログラミング" + 0.009*"勉強" + 0.007*"知識" + 0.007*"エンジニア" + 0.007*"学習" + 0.006*"考え" + 0.006*"内容" + 0.006*"について"
topic #4 (0.017): 0.080*"()" + 0.035*".s" + 0.031*"string" + 0.025*"if" + 0.020*"false" + 0.020*"true" + 0.017*"修正" + 0.016*"::" + 0.015*"int" + 0.014*"main"
topic #15 (0.017): 0.013*"使用" + 0.012*"可能" + 0.012*"について" + 0.012*"機能" + 0.010*"によって" + 0.008*"利用" + 0.00

topic #57 (0.017): 0.090*"クラス" + 0.080*"メソッド" + 0.048*"オブジェクト" + 0.034*"変数" + 0.031*"定義" + 0.024*"引数" + 0.021*"インスタンス" + 0.015*"プロパティ" + 0.013*"宣言" + 0.013*"使用"
topic #59 (0.017): 0.140*"アプリ" + 0.052*"android" + 0.032*"ios" + 0.023*"xcode" + 0.018*"ui" + 0.017*"対応" + 0.015*"app" + 0.014*"apple" + 0.014*"端末" + 0.013*"swift"
topic #58 (0.017): 0.102*"view" + 0.068*"xml" + 0.044*"spring" + 0.043*"model" + 0.043*"application" + 0.039*"アノテーション" + 0.033*"ビュー" + 0.028*"controller" + 0.027*"カード" + 0.026*"boot"
topic #41 (0.017): 0.086*"記事" + 0.035*"ください" + 0.020*"投稿" + 0.020*"さん" + 0.019*"について" + 0.016*"コメント" + 0.016*"こちら" + 0.015*"ござい" + 0.014*"qiita" + 0.014*"今回"
topic diff=0.014890, rho=0.120708
-11.844 per-word bound, 3675.4 perplexity estimate based on a held-out corpus of 1264 documents with 272188 words
PROGRESS: pass 19, at document #97264/97264
merging changes from 1264 documents into a model of 97264 documents
topic #46 (0.017): 0.035*"認証" + 0.034*"ユーザー" + 0.023*"ユーザ" + 0.022*"ログイン" 

In [145]:
lda_final.save("model/lda_final_5yeardic_2019.model")
get_perplexity(lda_final, corpus_5years)

saving LdaState object under model/lda_final_5yeardic_2019.model.state, separately None
saved model/lda_final_5yeardic_2019.model.state
saving LdaModel object under model/lda_final_5yeardic_2019.model, separately ['expElogbeta', 'sstats']
storing np array 'expElogbeta' to model/lda_final_5yeardic_2019.model.expElogbeta.npy
not storing attribute dispatcher
not storing attribute id2word
not storing attribute state
saved model/lda_final_5yeardic_2019.model


test size:  9726
train size:  87538


-11.741 per-word bound, 3422.4 perplexity estimate based on a held-out corpus of 9726 documents with 2051857 words


3422.387500346633


3422.387500346633

In [146]:
d.save("../data/dictionary/dict_final_14-19.dict")

saving Dictionary object under ../data/dictionary/dict_final_14-19.dict, separately None
saved ../data/dictionary/dict_final_14-19.dict


In [ ]:
def get_perplexity(ldamodel, corpus):
    test_size = int(len(corpus) * 0.1)
    test_corpus = corpus[:test_size]
    train_corpus= corpus[test_size:]
    print("test size: ", len(test_corpus))
    print("train size: ", len(train_corpus))
    
    perplexity = np.exp2(-ldamodel.log_perplexity(test_corpus))
    print(perplexity)
    return perplexity

get_perplexity(lda1, corpus__month)

## 五年分の辞書を使用

結果　perplexityが752

In [116]:
d = concat_dict("2014-2018")

loading Dictionary object from buf_dic/dict2014.dict
loaded buf_dic/dict2014.dict
discarding 134014 tokens: [('_・', 1), ('stringjoiner', 1), ('これ', 13504), ('たら', 10308), ('できる', 12825), ('ない', 22537), ('なる', 11490), ('まし', 14577), ('もの', 9893), ('cvfold', 1)]...
keeping 31305 tokens which were in no less than 5 and no more than 7744 (=20.0%) documents
resulting dictionary: Dictionary(31305 unique tokens: ['...', '02', '14', '2014', 'api']...)
loading Dictionary object from buf_dic/dict2015.dict
loaded buf_dic/dict2015.dict


31305
d_all: 31305


discarding 193515 tokens: [('ある', 25423), ('いる', 29348), ('から', 28996), ('こと', 30092), ('この', 20295), ('これ', 19824), ('する', 41622), ('そう', 10936), ('たい', 18400), ('たら', 15182)]...
keeping 42210 tokens which were in no less than 5 and no more than 10720 (=20.0%) documents
resulting dictionary: Dictionary(42210 unique tokens: ['issue', 'map', 'palette', 'sample', 'styleguide']...)
loading Dictionary object from buf_dic/dict2016.dict
loaded buf_dic/dict2016.dict


42210
d_all: 44881


discarding 249290 tokens: [('あり', 24059), ('ある', 32653), ('いる', 37833), ('から', 37312), ('これ', 24836), ('する', 51495), ('その', 19687), ('ため', 23866), ('だけ', 20951), ('でき', 29232)]...
keeping 51109 tokens which were in no less than 5 and no more than 12995 (=20.0%) documents
resulting dictionary: Dictionary(51109 unique tokens: ['1号', '2号', 'a.', 'basebackup', 'con']...)
loading Dictionary object from buf_dic/dict2017.dict
loaded buf_dic/dict2017.dict


51109
d_all: 57178


discarding 266550 tokens: [('ある', 36523), ('いる', 42060), ('から', 41521), ('この', 30539), ('する', 56536), ('でき', 33144), ('です', 41716), ('として', 26456), ('ない', 45378), ('なく', 22039)]...
keeping 55064 tokens which were in no less than 5 and no more than 13826 (=20.0%) documents
resulting dictionary: Dictionary(55064 unique tokens: ['ascii', 'bom', 'browsers', 'by', 'character']...)
loading Dictionary object from buf_dic/dict2018.dict
loaded buf_dic/dict2018.dict


55064
d_all: 66394


discarding 344145 tokens: [('("-"', 3), ('あり', 39903), ('ある', 52260), ('いる', 59391), ('から', 59130), ('ここ', 25066), ('こと', 62454), ('これ', 39463), ('する', 79309), ('そう', 23192)]...
keeping 68408 tokens which were in no less than 5 and no more than 19194 (=20.0%) documents
resulting dictionary: Dictionary(68408 unique tokens: ['10', '13.', '16', '2.0', '4r']...)


68408
d_all: 80216
80216


In [140]:
samp = ! ls ../data/article | grep 2019

df_2019month = pd.read_csv('../data/article/' + samp[0])
corpus_5_month = make_bow_corpus(d, df_2019month)

lda5 = models.ldamodel.LdaModel(corpus=corpus_5_month, id2word=d, num_topics=60, passes=20)

#corpora.MmCorpus.serialize("corpus/corpus_default_one_month_bow.mm", corpus_5_month)

[3792, 3846, 3638, 3989, 3946, 4283, 4056, 3953, 4236, 4501, 8492, 4868, 5117, 5316, 4807, 4819, 4812, 4863, 4604, 4897, 4992, 5056, 10584, 5112, 4901, 5274, 5222, 4979, 4871, 4707, 4713, 5030, 5447, 5948, 11777, 6263, 5958, 7180, 7179, 7296, 7364, 7099, 7231, 7296, 8439, 8246, 14399, 8287, 8225, 9322, 9481, 10056, 9899, 9762, 9508, 9839, 10453, 10656, 63]


using symmetric alpha at 0.016666666666666666
using symmetric eta at 0.016666666666666666
using serial LDA version on this node
running online (multi-pass) LDA training, 60 topics, 20 passes over the supplied corpus of 8225 documents, updating model once every 2000 documents, evaluating perplexity every 8225 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #14 (0.017): 0.006*"関数" + 0.005*"変更" + 0.004*"コマンド" + 0.004*"取得" + 0.004*"環境" + 0.004*"クリック" + 0.004*"選択" + 0.004*"キー" + 0.004*"api" + 0.004*"使用"
topic #18 (0.017): 0.008*"処理" + 0.007*"コード" + 0.006*"メソッド" + 0.005*"変更" + 0.005*"コメント" + 0.004*"データ" + 0.004*"書く" + 0.004*"関数" + 0.003*"実装" + 0.003*"みる"
topic #28 (0.017): 0.004*"データ" + 0.004*"コマンド" + 0.003*"問題" + 0.003*"取得" + 0.003*"処理" + 0.003*"今回" + 0.003*"アクセス" + 0.003*"削除" + 0.003*"web" + 0.003*"変更"
topic #48 (0.017): 0.006*"コード" + 0.005*"記事" + 0.005*"実装" +

KeyboardInterrupt: 

In [115]:
lda5.save("model/lda5year_1month.model")
get_perplexity(lda5, corpus_5_month)

saving LdaState object under model/lda5year_1month.model.state, separately None
saved model/lda5year_1month.model.state
saving LdaModel object under model/lda5year_1month.model, separately ['expElogbeta', 'sstats']
storing np array 'expElogbeta' to model/lda5year_1month.model.expElogbeta.npy
not storing attribute dispatcher
not storing attribute id2word
not storing attribute state
saved model/lda5year_1month.model


test size:  822
train size:  7403


-11.800 per-word bound, 3564.6 perplexity estimate based on a held-out corpus of 822 documents with 153659 words


3564.5610907855685


3564.5610907855685

## 一年分の辞書を使用

perplexityが4085

In [105]:
d = corpora.Dictionary.load("buf_dic/dict2018.dict")
d.filter_extremes(no_below=5, no_above=0.2)
d.compactify()

loading Dictionary object from buf_dic/dict2018.dict
loaded buf_dic/dict2018.dict
discarding 344145 tokens: [('("-"', 3), ('あり', 39903), ('ある', 52260), ('いる', 59391), ('から', 59130), ('ここ', 25066), ('こと', 62454), ('これ', 39463), ('する', 79309), ('そう', 23192)]...
keeping 68408 tokens which were in no less than 5 and no more than 19194 (=20.0%) documents
resulting dictionary: Dictionary(68408 unique tokens: ['10', '13.', '16', '2.0', '4r']...)


In [69]:
samp = ! ls ../data/article | grep 2019

df_2019month = pd.read_csv('../data/article/' + samp[0])
corpus__month = make_bow_corpus(d, df_2019month)

lda1 = models.ldamodel.LdaModel(corpus=corpus__month, id2word=d, num_topics=60, passes=10)

#corpora.MmCorpus.serialize("corpus/corpus_default_one_month_bow.mm", corpus_5_month)

[8225, 9322, 9481, 10056, 9899, 9762, 9508, 9839, 10453, 10656, 63]


using symmetric alpha at 0.016666666666666666
using symmetric eta at 0.016666666666666666
using serial LDA version on this node
running online (multi-pass) LDA training, 60 topics, 10 passes over the supplied corpus of 8225 documents, updating model once every 2000 documents, evaluating perplexity every 8225 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #46 (0.017): 0.005*"実装" + 0.003*"接続" + 0.003*"取得" + 0.003*"定義" + 0.003*"でしょ" + 0.003*"api" + 0.003*"ページ" + 0.003*"サイズ" + 0.003*"インスタンス" + 0.002*"関数"
topic #13 (0.017): 0.004*"クラス" + 0.004*"起動" + 0.004*"プロジェクト" + 0.004*"生成" + 0.004*"削除" + 0.003*"画像" + 0.003*"css" + 0.003*"エラー" + 0.003*"メソッド" + 0.003*"ユーザー"
topic #48 (0.017): 0.007*"関数" + 0.004*"プログラム" + 0.004*"情報" + 0.004*"64" + 0.004*"ーー" + 0.003*"要素" + 0.003*"生成" + 0.003*"プロジェクト" + 0.003*"クラス" + 0.003*"入力"
topic #53 (0.017): 0.004*"問題" + 0.003*"メッセージ" + 

topic #53 (0.017): 0.018*"起動" + 0.015*"ssh" + 0.015*"サーバ" + 0.014*"接続" + 0.013*"os" + 0.013*"linux" + 0.013*"ubuntu" + 0.013*"コマンド" + 0.010*"構築" + 0.010*"ホスト"
topic #18 (0.017): 0.035*"bit" + 0.029*"32" + 0.028*"64" + 0.027*"php" + 0.023*"28" + 0.011*"eclipse" + 0.011*"apache" + 0.011*"nginx" + 0.010*"14" + 0.008*"センサー"
topic #3 (0.017): 0.016*"azure" + 0.014*"ディレクトリ" + 0.012*"バージョン" + 0.010*"build" + 0.009*"vm" + 0.009*"マウント" + 0.009*"sdk" + 0.008*"コマンド" + 0.008*"起動" + 0.007*"file"
topic #20 (0.017): 0.021*"集合" + 0.013*")$" + 0.012*"}(" + 0.010*"cdots" + 0.010*"$\" + 0.010*"cf" + 0.008*"partial" + 0.008*"推論" + 0.008*"定義" + 0.007*"position"
topic diff=0.555141, rho=0.404474
-12.503 per-word bound, 5806.0 perplexity estimate based on a held-out corpus of 225 documents with 48139 words
PROGRESS: pass 1, at document #8225/8225
merging changes from 225 documents into a model of 8225 documents
topic #14 (0.017): 0.090*"移動" + 0.083*"--" + 0.037*"no" + 0.025*"2010" + 0.023*"mixin" + 0.020*"アセ

topic #13 (0.017): 0.100*"修正" + 0.034*"生成" + 0.029*"~]" + 0.024*"画像" + 0.020*"gan" + 0.018*"generator" + 0.018*"テスト" + 0.016*"アフィン" + 0.016*"潜在" + 0.016*"gc"
topic diff=0.165610, rho=0.351093
PROGRESS: pass 3, at document #6000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #55 (0.017): 0.033*"コマンド" + 0.016*"起動" + 0.012*"フォルダ" + 0.012*"ディレクトリ" + 0.012*"windows" + 0.011*"ダウンロード" + 0.010*"ましょ" + 0.009*"手順" + 0.009*"選択" + 0.009*"入力"
topic #12 (0.017): 0.116*"go" + 0.041*"oci" + 0.021*"oracle" + 0.018*"asset" + 0.017*"bigquery" + 0.017*"instance" + 0.013*"subnet" + 0.013*"契約" + 0.012*"<=" + 0.011*"import"
topic #23 (0.017): 0.055*"docker" + 0.036*"コンテナ" + 0.026*"イメージ" + 0.020*"db" + 0.016*"mysql" + 0.012*"laravel" + 0.011*"削除" + 0.010*"インスタンス" + 0.010*"php" + 0.009*"定義"
topic #50 (0.017): 0.030*"rfc" + 0.019*"for" + 0.017*"in" + 0.017*"protocol" + 0.015*"and" + 0.013*"radius" + 0.013*""," + 0.011*"network" + 0.009*"access" + 0.009*".["
topic #21 (0.017): 0.06

topic #50 (0.017): 0.019*"protocol" + 0.017*"for" + 0.016*"network" + 0.014*"and" + 0.013*"in" + 0.013*"rfc" + 0.012*"policy" + 0.010*"hack" + 0.010*"on" + 0.008*"the"
topic #53 (0.017): 0.030*"起動" + 0.025*"linux" + 0.019*"os" + 0.018*"ssh" + 0.018*"コマンド" + 0.016*"サーバ" + 0.015*"ユーザ" + 0.014*"接続" + 0.013*"ubuntu" + 0.011*"パスワード"
topic #33 (0.017): 0.044*"接続" + 0.022*"pi" + 0.015*"raspberry" + 0.014*"セッション" + 0.011*"資料" + 0.010*"pc" + 0.009*"カテゴリ" + 0.009*"動画" + 0.008*"音声" + 0.008*"android"
topic #12 (0.017): 0.114*"go" + 0.031*"oci" + 0.027*"oracle" + 0.021*"instance" + 0.018*"asset" + 0.015*"public" + 0.014*"vcn" + 0.013*"subnet" + 0.012*"golang" + 0.012*"契約"
topic diff=0.087385, rho=0.314464
PROGRESS: pass 5, at document #4000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #6 (0.017): 0.064*"csv" + 0.028*"pandas" + 0.025*"結合" + 0.020*"スキーマ" + 0.020*"人工知能" + 0.020*"一括" + 0.020*"統計" + 0.015*"join" + 0.013*"haskell" + 0.013*"出力"
topic #46 (0.017): 0.044*"カー

topic #58 (0.017): 0.027*"->" + 0.021*"平面" + 0.019*"mathbf" + 0.015*"}_" + 0.014*"text" + 0.013*"//" + 0.012*"in" + 0.011*")," + 0.010*"$\" + 0.009*"","
topic diff=0.052161, rho=0.299981
-12.141 per-word bound, 4515.9 perplexity estimate based on a held-out corpus of 225 documents with 48139 words
PROGRESS: pass 6, at document #8225/8225
merging changes from 225 documents into a model of 8225 documents
topic #17 (0.017): 0.038*"excel" + 0.033*"-1" + 0.032*"-2" + 0.028*"みんな" + 0.020*"センサ" + 0.019*"arduino" + 0.019*"-3" + 0.016*"マクロ" + 0.016*"access" + 0.015*"プロシージャ"
topic #12 (0.017): 0.099*"go" + 0.042*"oracle" + 0.033*"oci" + 0.026*"asset" + 0.021*"public" + 0.015*"instance" + 0.015*"bigquery" + 0.012*"golang" + 0.012*"import" + 0.011*"<="
topic #11 (0.017): 0.021*"twitter" + 0.020*"試験" + 0.015*"yy" + 0.014*"資格" + 0.013*"取得" + 0.012*"query" + 0.012*"api" + 0.011*"内容" + 0.011*"フォーク" + 0.011*"er"
topic #51 (0.017): 0.013*"リクエスト" + 0.012*"json" + 0.012*"html" + 0.011*"api" + 0.009*"js" +

topic #43 (0.017): 0.120*"google" + 0.038*"シート" + 0.036*"セル" + 0.024*"スプレッドシート" + 0.020*"script" + 0.017*"日本語" + 0.013*"drive" + 0.012*"apps" + 0.011*"英語" + 0.011*"スクリプト"
topic #56 (0.017): 0.053*"動画" + 0.046*"プロセス" + 0.021*"タグ" + 0.021*"::" + 0.018*"辞書" + 0.017*"pdf" + 0.015*"変換" + 0.013*"投稿" + 0.012*"mecab" + 0.011*"date"
topic #6 (0.017): 0.061*"csv" + 0.039*"pandas" + 0.027*"結合" + 0.020*"スキーマ" + 0.018*"一括" + 0.018*"統計" + 0.016*"人工知能" + 0.016*"join" + 0.013*"出力" + 0.012*"分析"
topic #33 (0.017): 0.047*"接続" + 0.023*"pi" + 0.018*"raspberry" + 0.017*"セッション" + 0.014*"資料" + 0.013*"デバイス" + 0.013*"pc" + 0.011*"android" + 0.011*"room" + 0.010*"カテゴリ"
topic diff=0.050471, rho=0.276158
PROGRESS: pass 8, at document #8000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #54 (0.017): 0.020*"by" + 0.019*"モデル" + 0.018*"投稿" + 0.017*"ユーザー" + 0.016*"テンソル" + 0.012*"スキル" + 0.012*"id" + 0.012*"user" + 0.011*"select" + 0.011*"model"
topic #52 (0.017): 0.051*"関数" + 0.026*"引数" + 

In [88]:
lda1.save("model/lda1year_1month.model")
get_perplexity(lda1, corpus__month)

saving LdaState object under model/lda1year_1month.model.state, separately None
saved model/lda1year_1month.model.state
saving LdaModel object under model/lda1year_1month.model, separately ['expElogbeta', 'sstats']
storing np array 'expElogbeta' to model/lda1year_1month.model.expElogbeta.npy
not storing attribute dispatcher
not storing attribute id2word
not storing attribute state
saved model/lda1year_1month.model


test size:  822
train size:  7403


-11.996 per-word bound, 4085.2 perplexity estimate based on a held-out corpus of 822 documents with 153659 words


4085.213609021161


4085.213609021161

# TFIDFを試す

78384537196.90767

632964060.034234


In [111]:
# tfidf
import gensim
import pickle
# # tfidf modelの生成
tfidf = gensim.models.TfidfModel(corpus_5_month)
tfidf.save('corpus/model_5month.tfidf')

# make corpus_tfidf
corpus_tfidf = tfidf[corpus_5_month]

# save corpus_tfidf
with open("corpus/corpus5_tfidf.pkl",'wb') as f:
    pickle.dump(corpus_tfidf,f)

collecting document frequencies
PROGRESS: processing document #0
calculating IDF weights for 8225 documents and 11262 features (739564 matrix non-zeros)
saving TfidfModel object under corpus/model_5month.tfidf, separately None
saved corpus/model_5month.tfidf


In [112]:
lda_tfidf = models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=d, num_topics=60, passes=10)

using symmetric alpha at 0.016666666666666666
using symmetric eta at 0.016666666666666666
using serial LDA version on this node
running online (multi-pass) LDA training, 60 topics, 10 passes over the supplied corpus of 8225 documents, updating model once every 2000 documents, evaluating perplexity every 8225 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #34 (0.017): 0.002*"書き" + 0.002*"修正" + 0.002*"php" + 0.002*"開放" + 0.002*"端末" + 0.002*"実機" + 0.001*"example" + 0.001*"再現" + 0.001*"取り扱い" + 0.001*"moment"
topic #49 (0.017): 0.002*"現在" + 0.002*"不愉快" + 0.002*"もと" + 0.001*"property" + 0.001*"離さ" + 0.001*"ダイジェスト" + 0.001*"date" + 0.001*"飛ん" + 0.001*"sqlインジェクション" + 0.001*"特定非営利活動法人"
topic #15 (0.017): 0.002*"発想の転換" + 0.002*"affordable" + 0.002*"using" + 0.002*"ラスボス" + 0.001*"取り扱い" + 0.001*"任意" + 0.001*"文字列" + 0.001*"中身" + 0.001*"たった" + 0.001*"文明"
topic #42 (0.0

PROGRESS: pass 1, at document #8000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #56 (0.017): 0.017*"rfc" + 0.011*"どこ" + 0.005*"cards" + 0.005*"自動" + 0.005*"rad" + 0.005*"成長" + 0.005*"合格" + 0.005*"あの" + 0.004*"フレームワーク" + 0.004*"最中"
topic #25 (0.017): 0.008*"環境変数" + 0.007*"2.0" + 0.007*"行わ" + 0.007*"store" + 0.006*"マップ" + 0.006*"じゃなくて" + 0.006*"プロセス" + 0.005*"伝播" + 0.005*"解説" + 0.004*"コミュニケーション"
topic #43 (0.017): 0.004*"over" + 0.004*"local" + 0.003*"saving" + 0.002*"魔改造" + 0.002*"冷静" + 0.002*"<>" + 0.001*"/." + 0.001*"本体" + 0.001*"カラム名" + 0.001*"sum"
topic #23 (0.017): 0.008*"浮かび上がっ" + 0.006*"eclipse" + 0.005*"厳しく" + 0.005*"fabric" + 0.004*"あきらめ" + 0.004*"届く" + 0.004*"かける" + 0.004*"手引き" + 0.004*"定か" + 0.003*"率い"
topic #45 (0.017): 0.013*"29" + 0.009*"?/" + 0.008*"ほめ" + 0.008*"2月" + 0.006*"suit" + 0.006*"xampp" + 0.005*"おろし" + 0.005*"20代" + 0.004*"suitable" + 0.004*"welcome"
topic diff=0.070157, rho=0.404474
-24.261 per-word bound, 20105753.4 perplexity

topic #2 (0.017): 0.002*"e's" + 0.002*"ドラマ" + 0.001*"1999" + 0.001*"でかく" + 0.000*"サクッと" + 0.000*"おつかれさま" + 0.000*"three" + 0.000*"アルミ" + 0.000*"discouraged" + 0.000*"rover"
topic #18 (0.017): 0.014*"バー" + 0.007*"iterator" + 0.006*"ジェネレーター" + 0.006*"性能" + 0.004*"hope" + 0.004*"分け" + 0.003*"runnable" + 0.003*"easy" + 0.003*"放す" + 0.003*"ブーツ"
topic #49 (0.017): 0.014*"目次" + 0.005*"すなわち" + 0.004*"schedule" + 0.004*"白丸" + 0.004*"クリップボード" + 0.003*"定義書" + 0.003*"ダイジェスト" + 0.003*"linux" + 0.002*"wife" + 0.002*"pypi"
topic #45 (0.017): 0.017*"29" + 0.015*"?/" + 0.011*"ほめ" + 0.008*"2月" + 0.007*"20代" + 0.006*"おろし" + 0.006*"began" + 0.006*"suit" + 0.006*"xampp" + 0.005*"リテラル"
topic diff=0.012689, rho=0.351093
PROGRESS: pass 3, at document #6000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #59 (0.017): 0.006*"手持ち" + 0.006*"まず" + 0.003*"menuitem" + 0.003*"のみ" + 0.002*"伸縮自在" + 0.002*"semantic" + 0.001*"bus" + 0.001*"インド" + 0.001*"show" + 0.001*"空腹"
topic #51 (0.017): 

topic #59 (0.017): 0.009*"手持ち" + 0.004*"まず" + 0.002*"のみ" + 0.002*"menuitem" + 0.001*"伸縮自在" + 0.001*"semantic" + 0.001*"bus" + 0.001*"show" + 0.001*"空腹" + 0.000*"インド"
topic #49 (0.017): 0.025*"目次" + 0.007*"クリップボード" + 0.004*"定義書" + 0.004*"ll" + 0.004*"白丸" + 0.004*"すなわち" + 0.003*"ダイジェスト" + 0.002*"schedule" + 0.002*"linux" + 0.002*"えらぶ"
topic diff=0.013417, rho=0.331269
PROGRESS: pass 5, at document #2000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #32 (0.017): 0.007*"解答" + 0.007*"考えろ" + 0.006*"じゃあ" + 0.002*"基本情報" + 0.002*"じん" + 0.002*"お喋り" + 0.002*"購入" + 0.002*"違和感" + 0.002*"面倒" + 0.002*"今にも"
topic #43 (0.017): 0.005*"local" + 0.003*"over" + 0.003*"/." + 0.003*"カラム名" + 0.002*"sum" + 0.002*"saving" + 0.002*"魔改造" + 0.001*"冷静" + 0.001*"<>" + 0.001*"小規模"
topic #35 (0.017): 0.008*"lan" + 0.007*"すごく" + 0.007*"100万" + 0.005*"強み" + 0.005*"microsoft" + 0.005*"わかり" + 0.005*"途方" + 0.005*"less" + 0.005*"ましょ" + 0.004*"応用"
topic #9 (0.017): 0.003*"restaurant" + 0.003*"

PROGRESS: pass 6, at document #8000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #37 (0.017): 0.004*"寝坊" + 0.004*"動かす" + 0.004*"よかっ" + 0.004*"詳しい" + 0.003*"サクッと" + 0.003*"さて" + 0.003*"androidmanifest" + 0.003*"変われ" + 0.002*"逼迫" + 0.002*"eve"
topic #20 (0.017): 0.008*"touchdesigner" + 0.006*"同時に" + 0.006*"gr" + 0.005*"キーボード" + 0.005*"bison" + 0.005*"読め" + 0.004*"お話" + 0.004*"シミュレータ" + 0.003*"vpn" + 0.003*"有名"
topic #27 (0.017): 0.003*"低く" + 0.002*"user" + 0.002*"わー" + 0.001*"absence" + 0.000*"ニューヨーク" + 0.000*"勤務" + 0.000*"当日" + 0.000*"float" + 0.000*"hour" + 0.000*"統一"
topic #2 (0.017): 0.003*"ドラマ" + 0.002*"e's" + 0.002*"1999" + 0.001*"でかく" + 0.000*"サクッと" + 0.000*"おつかれさま" + 0.000*"three" + 0.000*"アルミ" + 0.000*"discouraged" + 0.000*"rover"
topic #18 (0.017): 0.017*"バー" + 0.006*"runnable" + 0.006*"iterator" + 0.005*"ジェネレーター" + 0.005*"性能" + 0.004*"hope" + 0.003*"amazing" + 0.003*"ダブル" + 0.002*"分け" + 0.002*"ブーツ"
topic diff=0.003820, rho=0.299981
-24.137 per-

topic #1 (0.017): 0.009*"レッスン" + 0.003*"要因" + 0.003*"解決" + 0.003*"邦題" + 0.002*"毎月" + 0.002*"es" + 0.002*"明らか" + 0.002*"財産" + 0.002*"continuously" + 0.002*"明文化"
topic #41 (0.017): 0.005*"おいで" + 0.004*"限定" + 0.003*"プレビュー" + 0.003*"まとまっ" + 0.003*"ころ" + 0.003*"家族" + 0.002*"のる" + 0.002*"hair" + 0.002*"家庭" + 0.002*"destruction"
topic #56 (0.017): 0.026*"rfc" + 0.013*"どこ" + 0.009*"自動" + 0.008*"成長" + 0.006*"傾向" + 0.006*"ネタ" + 0.005*"フレームワーク" + 0.005*"合格" + 0.004*"tail" + 0.004*"その間"
topic #9 (0.017): 0.004*"restaurant" + 0.003*"wine" + 0.003*"30分" + 0.003*"words" + 0.003*"違う" + 0.002*"スネークケース" + 0.002*"onclick" + 0.002*"雇っ" + 0.002*"さわる" + 0.002*"void"
topic diff=0.003558, rho=0.276158
PROGRESS: pass 8, at document #6000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #33 (0.017): 0.005*"river" + 0.003*"プラス" + 0.003*"chrome" + 0.003*"御礼" + 0.003*"chime" + 0.003*"復習" + 0.002*"つまみ" + 0.002*"挙がり" + 0.002*"おしまい" + 0.001*"46"
topic #18 (0.017): 0.017*"バー" + 0.007*"iter

topic #42 (0.017): 0.007*"スッキリ" + 0.003*"ありません" + 0.002*"interface" + 0.002*"切羽詰まっ" + 0.002*"th" + 0.002*"見つける" + 0.002*"測定" + 0.002*"批評家" + 0.001*"ふえ" + 0.001*"トライアル版"
topic diff=0.009220, rho=0.266194


In [113]:
lda_tfidf.save("model/ldatfidf_1year_1month.model")
get_perplexity(lda_tfidf, corpus_tfidf)

saving LdaState object under model/ldatfidf_1year_1month.model.state, separately None
saved model/ldatfidf_1year_1month.model.state
saving LdaModel object under model/ldatfidf_1year_1month.model, separately ['expElogbeta', 'sstats']
storing np array 'expElogbeta' to model/ldatfidf_1year_1month.model.expElogbeta.npy
not storing attribute dispatcher
not storing attribute id2word
not storing attribute state
saved model/ldatfidf_1year_1month.model


test size:  822
train size:  7403


-29.238 per-word bound, 632964060.0 perplexity estimate based on a held-out corpus of 822 documents with 5071 words


632964060.034234


632964060.034234

In [ ]:
def get_topic_words(topic_id, lda):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))
for t in range(10):
    print("Topic # ",t)
    get_topic_words(t, lda5)
    print("\n")

# 可視化

perplexity: 2488.382900069194
5years perplexity: perplexity: 1981.2039144482053

# 保存

In [185]:
# d.save_as_text("data/dict.txt")
# corpora.MmCorpus.serialize("data/cop.mm", corpus)
lda5.save("data/lda5.model")
# doc_index.save("data/sim")

saving LdaState object under data/lda5.model.state, separately None
saved data/lda5.model.state
saving LdaModel object under data/lda5.model, separately ['expElogbeta', 'sstats']
storing np array 'expElogbeta' to data/lda5.model.expElogbeta.npy
not storing attribute id2word
not storing attribute dispatcher
not storing attribute state
saved data/lda5.model


In [1]:
from gensim import models, corpora, similarities

corpus = corpora.MmCorpus("data/cop.mm")
lda = models.ldamodel.LdaModel.load("data/lda.model")
d = corpora.Dictionary.load_from_text("data/dict.txt")
doc_index = similarities.docsim.MatrixSimilarity.load("data/sim")

In [2]:
doc_index.__getitem__(lda[corpus[0]])

array([0.99999994, 0.03107498, 0.07525122, ..., 0.03280416, 0.29749227,
       0.05943879], dtype=float32)